In [1]:
using Rx
using BenchmarkTools

In [2]:
struct KeepActor{D} <: NextActor{D} 
    values::Vector{D}
    
    KeepActor{D}() where D = new(Vector{D}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

In [3]:
n = 500

array      = [ i for i in 1:n ]
observable = from(array);

In [4]:
reactive(source::S) where S = reactive(as_subscribable(S), source)

function reactive(::ValidSubscribable{T}, source) where T
    keep = KeepActor{T}()
    subscribe!(source, keep)
    return keep.values
end

reactive (generic function with 2 methods)

# Map operator performance comparison

In [5]:
function regularArrayMap(source)
    keep = Vector{Int}()
    for v in source
        push!(keep, v ^ 2)
    end
    return keep
end

@CreateMapOperator(SquaredInt, Int, Int, (d) -> d ^ 2)

observable_mapped1 = observable |> SquaredIntMapOperator()
observable_mapped2 = observable |> map(Int, (d) -> d ^ 2)

println(regularArrayMap(array) == reactive(observable_mapped1))
println(regularArrayMap(array) == reactive(observable_mapped2))

@btime regularArrayMap(array);
@btime reactive(observable_mapped1);
@btime reactive(observable_mapped2);

true
true
  3.090 μs (9 allocations: 8.33 KiB)
  3.431 μs (11 allocations: 8.36 KiB)
  35.719 μs (989 allocations: 23.66 KiB)


# Filter operator performance comparison

In [6]:
function regularArrayFilter(source)
    keep   = Vector{Int}()
    for v in source
        if v % 2 == 0
            push!(keep, v)
        end
    end
    return keep
end

@CreateFilterOperator(EvenInt, Int, (d) -> d % 2 == 0)

observable_filtered1 = observable |> EvenIntFilterOperator()
observable_filtered2 = observable |> filter((d) -> d % 2 == 0)

println(regularArrayFilter(array) == reactive(observable_filtered1))
println(regularArrayFilter(array) == reactive(observable_filtered2))

@btime regularArrayFilter(array);
@btime reactive(observable_filtered1);
@btime reactive(observable_filtered2);

true
true
  1.851 μs (8 allocations: 4.27 KiB)
  2.039 μs (10 allocations: 4.30 KiB)
  23.880 μs (510 allocations: 12.13 KiB)


# Count operator performance comparison

In [7]:
function regularArrayCount(source)
    keep    = Vector{Int}()
    current = 0
    for v in source
        current += 1
    end
    push!(keep, current)
    return keep
end

observable_counted = observable |> count()

println(regularArrayCount(array) == reactive(observable_counted))

@btime regularArrayCount(array);
@btime reactive(observable_counted);

true
  61.221 ns (2 allocations: 128 bytes)
  256.893 ns (4 allocations: 176 bytes)


# Enumerate operator performance comparison

In [8]:
function regularArrayEnumerate(source)
    keep   = Vector{Tuple{Int, Int}}()
    current = 1
    for v in source
        push!(keep, (v, current))
        current += 1
    end
    return keep
end

observable_enumerated = observable |> enumerate()

println(regularArrayEnumerate(array) == reactive(observable_enumerated))

@btime regularArrayEnumerate(array);
@btime reactive(observable_enumerated);

true
  3.803 μs (9 allocations: 16.34 KiB)
  4.211 μs (11 allocations: 16.39 KiB)


# Scan operator performance comparison

In [9]:
function regularArrayScan(source)
    keep    = Vector{Int}()
    current = 0
    for v in source
        current = current + v
        push!(keep, current)
    end
    return keep
end

@CreateScanOperator(Sum, Int, Int, +)

observable_scanned1 = observable |> SumScanOperator(0)
observable_scanned2 = observable |> scan(Int, (d, c) -> d + c, 0)

println(regularArrayScan(array) == reactive(observable_scanned1))
println(regularArrayScan(array) == reactive(observable_scanned2))

@btime regularArrayScan(array);
@btime reactive(observable_scanned1);
@btime reactive(observable_scanned2);

true
true
  3.182 μs (9 allocations: 8.33 KiB)
  3.706 μs (11 allocations: 8.38 KiB)
  486.425 μs (3822 allocations: 83.56 KiB)


# Reduce operator performance comparison

In [10]:
function regularArrayReduce(source)
    keep    = Vector{Int}()
    current = 0
    for v in source
        current = current + v
    end
    push!(keep, current)
    return keep
end

@CreateReduceOperator(Sum, Int, Int, +)

observable_reduced1 = observable |> SumReduceOperator(0)
observable_reduced2 = observable |> reduce(Int, (d, c) -> d + c, 0)

println(regularArrayReduce(array) == reactive(observable_reduced1))
println(regularArrayReduce(array) == reactive(observable_reduced2))

@btime regularArrayReduce(array);
@btime reactive(observable_reduced1);
@btime reactive(observable_reduced2);

true
true
  87.531 ns (2 allocations: 128 bytes)
  283.650 ns (4 allocations: 176 bytes)
  466.394 μs (3815 allocations: 75.36 KiB)


# Max operator performance comparison

In [11]:
function regularArrayMax(source)
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            if v > current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

observable_max = observable |> max()

println(regularArrayMax(array) == reactive(observable_max))

@btime regularArrayMax(array);
@btime reactive(observable_max);

true
  550.840 ns (2 allocations: 128 bytes)
  857.429 ns (4 allocations: 176 bytes)


# Min operator performance comparison

In [12]:
function regularArrayMin(source)
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            if v < current
                current = v
            end
        end
    end
    push!(keep, current)
    return keep
end

observable_min = observable |> min()

println(regularArrayMin(array) == reactive(observable_min))

@btime regularArrayMin(array);
@btime reactive(observable_min);

true
  1.021 μs (2 allocations: 128 bytes)
  1.238 μs (4 allocations: 176 bytes)


# Sum operator performance comparison

In [13]:
function regularArraySum(source)
    keep    = Vector{Int}()
    current = nothing
    for v in source
        if current == nothing
            current = v
        else
            current = current + v
        end
    end
    push!(keep, current)
    return keep
end

observable_sum = observable |> sum()

println(regularArraySum(array) == reactive(observable_sum))

@btime regularArraySum(array);
@btime reactive(observable_sum);

true
  660.994 ns (2 allocations: 128 bytes)
  972.412 ns (4 allocations: 176 bytes)
